# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig

from azureml.widgets import RunDetails

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mental-health-classification'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EP5G6WL5Y to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


### Creating the compute

In [3]:
cpu_cluster_name = "cpu-cluster"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=7)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Data Preparation
#### Access the data

In [4]:
# Create a project_folder if it doesn't exist
if not os.path.isdir('data'):
    os.mkdir('data')
    
if not os.path.exists('project_folder'):
    os.makedirs('project_folder')



In [18]:
project_folder="./project_folder/"

In [5]:
df = pd.read_csv("Train.csv")
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
label = "label"

In [6]:
train_data.head()

,ID,text,label
56,6T8HVMG3,what can I do to minimize my alcoholism,Alcohol
335,458NTYKE,"Had a problem with my personal looks,feel sad",Depression
439,43COYA4O,What is the benefit of alcohol,Alcohol
292,V0C7LKYN,"I am very nervous and bothered,",Depression
611,BOHSNXCN,What should I do to stop alcoholism?,Alcohol


In [51]:
train=train_data.iloc[:, 1:]
test=test_data.iloc[:, 1:]


train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [52]:
train.head()

,text,label
56,what can I do to minimize my alcoholism,Alcohol
335,"Had a problem with my personal looks,feel sad",Depression
439,What is the benefit of alcohol,Alcohol
292,"I am very nervous and bothered,",Depression
611,What should I do to stop alcoholism?,Alcohol


In [53]:
datastore=ws.get_default_datastore()

In [55]:
datastore.upload(src_dir="./data", target_path="mental_health_clf", show_progress=True)

Uploading an estimated of 2 files
Uploading ./data/test.csv
Uploaded ./data/test.csv, 1 files out of an estimated total of 2
Uploading ./data/train.csv
Uploaded ./data/train.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_5faee4741be7409f856140a97920a633

In [56]:
train_set =TabularDatasetFactory.from_delimited_files(path=datastore.path("mental_health_clf/train.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [57]:
# TODO: Put your automl settings here
import time
automl_settings = {
    "name": "AutoML_mental_health".format(time.time()),
    "enable_early_stopping" : True,
    "experiment_timeout_minutes" : 40,
    "iteration_timeout_minutes": 10,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "max_concurrent_iterations": 10,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                             training_data=train_set,
                             label_column_name="label",
                             compute_target=cpu_cluster,
                             debug_log='automl_errors.log',
                             path=project_folder,
                             model_explainability=True,
                             **automl_settings,
                            )

In [58]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_690611a3-524f-4486-9e61-36a801aa4da6

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------------------

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [62]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET', …

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [63]:
best_run, fitted_model= remote_run.get_output()

In [64]:
print(best_run)

Run(Experiment: mental-health-classification,
Id: AutoML_690611a3-524f-4486-9e61-36a801aa4da6_48,
Type: azureml.scriptrun,
Status: Completed)


In [65]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               min_child_weight=1,
                                                                                               missing=nan,
                                                                                               n_estimators=25,
                                                         

In [66]:
fitted_model.named_steps["datatransformer"].get_engineered_feature_names()

['text_CharGramTfIdf_ , ',
 'text_CharGramTfIdf_ ,h',
 'text_CharGramTfIdf_ ,n',
 'text_CharGramTfIdf_ -d',
 'text_CharGramTfIdf_ -h',
 'text_CharGramTfIdf_ a ',
 'text_CharGramTfIdf_ ab',
 'text_CharGramTfIdf_ ac',
 'text_CharGramTfIdf_ ad',
 'text_CharGramTfIdf_ af',
 'text_CharGramTfIdf_ ag',
 'text_CharGramTfIdf_ ah',
 'text_CharGramTfIdf_ al',
 'text_CharGramTfIdf_ am',
 'text_CharGramTfIdf_ an',
 'text_CharGramTfIdf_ ap',
 'text_CharGramTfIdf_ ar',
 'text_CharGramTfIdf_ as',
 'text_CharGramTfIdf_ at',
 'text_CharGramTfIdf_ av',
 'text_CharGramTfIdf_ aw',
 'text_CharGramTfIdf_ ba',
 'text_CharGramTfIdf_ be',
 'text_CharGramTfIdf_ bh',
 'text_CharGramTfIdf_ bo',
 'text_CharGramTfIdf_ br',
 'text_CharGramTfIdf_ bu',
 'text_CharGramTfIdf_ by',
 'text_CharGramTfIdf_ ca',
 'text_CharGramTfIdf_ ce',
 'text_CharGramTfIdf_ ch',
 'text_CharGramTfIdf_ cl',
 'text_CharGramTfIdf_ co',
 'text_CharGramTfIdf_ cr',
 'text_CharGramTfIdf_ cu',
 'text_CharGramTfIdf_ d ',
 'text_CharGramTfIdf_ da',
 

In [67]:
fitted_model.named_steps["datatransformer"].get_featurization_summary()

[{'RawFeatureName': 'text',
  'TypeDetected': 'Text',
  'Dropped': 'No',
  'EngineeredFeatureCount': 5090,
  'Transformations': ['StringCast-CharGramTfIdf', 'StringCast-WordGramTfIdf']}]

In [68]:
fitted_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('8',
                                             Pipeline(memory=None,
                                                      steps=[('sparsenormalizer',
                                                              <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7fdf4a4aef28>),
                                                             ('xgboostclassifier',
                                                              XGBoostClassifier(base_score=0.

In [69]:
test.columns

Index(['text', 'label'], dtype='object')

In [70]:
from sklearn.metrics import accuracy_score

y_pred=fitted_model.predict(test.iloc[:, :1])

print("Model accuracy: ", accuracy_score(test['label'], y_pred))

Model accuracy:  0.7580645161290323


In [72]:
test=pd.read_csv("./Test.csv")
test.head()

,ID,text
0,02V56KMO,How to overcome bad feelings and emotions
1,03BMGTOK,I feel like giving up in life
2,03LZVFM6,I was so depressed feel like got no strength t...
3,0EPULUM5,I feel so low especially since I had no one to...
4,0GM4C5GD,can i be successful when I am a drug addict?


In [74]:
y_proba=fitted_model.predict_proba(test[["text"]])

In [75]:
y_proba

,Alcohol,Depression,Drugs,Suicide
0,0.09,0.71,0.02,0.17
1,0.01,0.95,0.01,0.02
2,0.01,0.96,0.01,0.02
3,0.02,0.90,0.02,0.06
4,0.17,0.38,0.25,0.20
...,...,...,...,...
304,0.16,0.52,0.16,0.16
305,0.05,0.58,0.06,0.31
306,0.30,0.51,0.05,0.14
307,0.03,0.04,0.88,0.04


In [76]:
sub=pd.DataFrame()

In [77]:
sub["ID"]=test.ID
sub["Depression"]=y_proba.Depression
sub["Alcohol"]=y_proba.Alcohol
sub["Suicide"]=y_proba.Suicide
sub["Drugs"]=y_proba.Drugs

In [78]:
sub.head()

,ID,Depression,Alcohol,Suicide,Drugs
0,02V56KMO,0.71,0.09,0.17,0.02
1,03BMGTOK,0.95,0.01,0.02,0.01
2,03LZVFM6,0.96,0.01,0.02,0.01
3,0EPULUM5,0.90,0.02,0.06,0.02
4,0GM4C5GD,0.38,0.17,0.20,0.25


In [79]:
sub.to_csv("./automl_1.csv", index=False)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service